In [5]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv('.env')
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [38]:
sys_msg = """ 
You are a helpful bilingual tutor who is helping a 7-10 years old student learning a new language. 
Strictly take the following steps before you answer the student's question:

1- Identify the native language of the student.
2- Identify the intent of the student's question:
    - Are they asking to practice the language they are learning through conversation?
    - Are they asking for help to learn the langugage?
3- Generate output based on the identified intent of the student's question.
    - If the student is asking for help to learn the language, output your answer in their native language.
    - If the student is asking to practice the language they are learning through conversation, output your answer in the language they are learning.

Your tone should be friendly and encouraging.

Now continue the conversation with the student, and strictly only output the answer to the student's question without explaining the thought process of the above steps.
"""

In [48]:
# Initialize conversation history
conversation_history = []

# Function to update conversation history
def update_conversation_history(user_input, model_response):
    # Store user input and model response as separate entries with their respective roles
    conversation_history.append({"role": "user", "content": user_input})
    conversation_history.append({"role": "assistant", "content": model_response})

# Function to construct the system message and combine it with the conversation history
def get_conversation_messages():
    # The system message that remains consistent throughout the conversation
    system_message = {"role": "system", "content": sys_msg}
    
    # Return the system message along with the conversation history formatted for the API
    # Limit the conversation history to the last 3 message pairs (user input, assistant response)
    return [system_message] + conversation_history[-6:]

# Function to query GPT-4 with user input and conversation history
def query_gpt(user_input):
    try:
        # Get the conversation history formatted as API messages, including the system message
        conversation_messages = get_conversation_messages()
        
        # Add the user's new input to the conversation
        conversation_messages.append({"role": "user", "content": user_input})
        
        # Make the API call to OpenAI's GPT model
        response = client.chat.completions.create(
            messages=conversation_messages,
            model="gpt-4",
        )
        # Extract the model's response
        model_response = response.choices[0].message.content
        
        # Update conversation history with the model's response
        update_conversation_history(user_input, model_response)
        
        return model_response
    except Exception as e:
        return f"Error: {str(e)}"

In [51]:
user_input = "你能教我怎么用spanish说吗"
print("User:", user_input)
response = query_gpt(user_input)
print("ChatGPT:", response)

User: 你能教我怎么用spanish说吗
ChatGPT: 当然可以，你可以说：“Me gusta comer pato laqueado de Pekín.”


In [ ]:


# Function to update conversation history
def update_conversation_history(user_input, model_response):
    conversation_history.append(f"User: {user_input}")
    conversation_history.append(f"AI: {model_response}")

# Function to construct the system prompt dynamically based on history
def get_system_prompt():
    history_prompt = "\n".join(conversation_history)
    return f"You are ChatGPT. Here is the conversation history:\n{history_prompt}"

# Function to query GPT-4 with user input and conversation history
def query_gpt(user_input):
    system_prompt = get_system_prompt()  # Construct prompt with history
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "system", "content": system_prompt},
                      {"role": "user", "content": user_input}]
        )
        # Extract the model's response
        model_response = response['choices'][0]['message']['content']
        # Update conversation history
        update_conversation_history(user_input, model_response)
        return model_response
    except Exception as e:
        return f"Error: {str(e)}"

# Example function to start a conversation
def start_conversation():
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = query_gpt(user_input)
        print(f"AI: {response}")

# Main execution: starts a conversation loop
if __name__ == "__main__":
    print("Start chatting with ChatGPT! Type 'exit' to quit.")
    start_conversation()
